<a href="https://colab.research.google.com/github/jouvetg/igm/blob/main/notebooks/IGM_paleo_alps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <h1 align="center" id="title">IGM set-up for paleo runs in the Alps</h1>

This notebook gives a simple set-up to run a paleo glacier model in the Alps in paleo times with different catchements with IGM (the Instructed Glacier Model). Check at https://github.com/jouvetg/igm for more info. MAKE SURE that you have activated a GPU (click on Runtime -> Change Runtime type -> select GPU)

First choose your areas among lyon, ticino, rhine, linth glaciers and choose the spatial resolution  (keep it coarse like 2000 for testing)

In [6]:
!pip install importlib-metadata==4.13.0

area = 'linth' # chose your area among lyon, linth, rhine, ticino
reso = 2000    # chose your resolution among 200,1000, 2000 meters (keep it coarse like 2000 for testing)

First we download the data

In [ ]:
# clean old staff if any
!rm -r f12_cfsflow_GJ_21_a f14_pism_GJ_21_a  __pycache__ data-for-paleo-tuto
!rm  *.nc  *.py *.dat *.txt *.zip

!apt install subversion

# get the core source of igm (in igm.py)
!wget -O igm.py https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py

# get the data (topography of the area + climate signal dta file)
!svn export https://github.com/jouvetg/igm/trunk/examples/paleo-alps/data-for-paleo-tuto

# get the ice flow DL-emulator at 1 and 2 km
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f14_pism_GJ_21_a

# get the ice flow DL-emulator at 100 and 200 meter
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f12_cfsflow_GJ_21_a

Let's import necessary libraries (you may also check what GPU you have been attributed, usually old slow K80 GPU with free colab, or fast T100 with the pro version, which costs 10 $ / months).

In [2]:
import tensorflow as tf
import math
from igm import Igm
import numpy as np
from scipy.interpolate import interp1d
from netCDF4 import Dataset
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt
import sys

sys.argv = ['']  # this is absolutly necessary for runs in notebooks

# Uncomment this if oyu want to check what GPU you are working on
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

To force the surface mass balance (SMB), we use the EPICA climate signal, and parametize the ELA as function of the EPICA Temperature offset (Delta T or dT). Let's open the data file, and plot the raw temperature deltat T, as well as an example EPICA-based ELA between 30 ka BP and 20 ky BP over time (here we took ELA = 3000 at present day and is decreased by 200 meters per °C, i.e. ELA = 1000 i.e. DT = -10 for illustation):


In [ ]:
ss = np.loadtxt('data-for-paleo-tuto/EDC_dD_temp_estim.tab',dtype=np.float32,skiprows=31)
time = ss[:,1] * -1000  # extract time BP, chnage unit to yeat
dT   = ss[:,3]          # extract the dT, i.e. global temp. difference
dT =  interp1d(time,dT, fill_value=(dT[0], dT[-1]),bounds_error=False)
TIME = np.arange(-30000,-15000,100) # discretize time between 30 and 20 ka BP with century steps
 
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(TIME, dT(TIME),'-r') 
ax2.plot(TIME, 3000 + 200.0*dT(TIME),'b') 
ax1.set_xlabel('Time')
ax1.set_ylabel('DT', color='g')
ax2.set_ylabel('ELA', color='b')
plt.show()

Let's define our own mass balance function, which defined ELA as function of EPICA's DT (ELA is low close to LGM, and high before / after), and the mass balance as function of ELA and other parameters. This mass balance function is simply advocated to the python class igm as it is not available in the original igm source code.

In [ ]:
class Igm(Igm):

    def init_smb_signal(self):
        """
            Retrieve the Temperature difference from the EPICA signal
        """
        # load the EPICA signal from the official data
        ss = np.loadtxt('data-for-paleo-tuto/EDC_dD_temp_estim.tab',dtype=np.float32,skiprows=31)
        time = ss[:,1] * -1000  # extract time BP, chnage unit to yeat
        dT   = ss[:,3]          # extract the dT, i.e. global temp. difference
        self.dT =  interp1d(time,dT, fill_value=(dT[0], dT[-1]),bounds_error=False)

    def update_smb_signal(self):
        """
            mass balance 'signal'
        """

        # this serves to open the EPICA file (this is done only once at start)
        if not hasattr(self, "already_called_update_smb_signal"):
            self.init_smb_signal()
            self.already_called_update_smb_signal = True

        # define ELA as function of EPICA's Delta T, ELA's present day (pdela) and Dela/Dt (deladt)
        ela     = self.config.pdela + self.config.deladt*self.dT(self.t) # for rhine

        # that's SMB param with ELA, ablation and accc gradient, and max accumulation
        # i.e. SMB =       gradabl*(z-ela)           if z<ela, 
        #          =  min( gradacc*(z-ela) , maxacc) if z>ela.
        smb = self.usurf - ela
        smb *= tf.where(tf.less(smb, 0), self.config.gradabl, self.config.gradacc)
        smb = tf.clip_by_value(smb, -100, self.config.maxacc)

        self.smb.assign( smb )

glacier = Igm()


Let's now define some pararameters, and run the code


In [ ]:

glacier.config.geology_file            = 'data-for-paleo-tuto/'+area+'-'+str(reso)+'.nc' 
glacier.config.tstart                  = -30000 
glacier.config.tend                    = -29000
glacier.config.tsave                   = 100  # save result each 100 years
glacier.config.cfl                     = 0.25 # if you see numerical unstabilities, decrease this.
glacier.config.init_strflowctrl        = 100  # this parameter controls the ice flow strengh, if you increase it, you accelerate the ice

# here you can change parameters that parametrize ELA 
glacier.config.pdela  = 3000 # Present-day ELA
if area=='rhine':
  glacier.config.deladt = 190 
elif area=='linth':
  glacier.config.deladt = 220 
else:
  glacier.config.deladt = 200

# here you can change parameters that parametrize the mass balance function (as defined above)
glacier.config.gradabl = 0.0067   # taken from (Cohen, 2018, TC)
glacier.config.gradacc = 0.0005   # taken from (Cohen, 2018, TC)
glacier.config.maxacc  = 1.0      # taken from (Cohen, 2018, TC)

if reso<=200:
  glacier.config.iceflow_model_lib_path  = 'f12_cfsflow_GJ_21_a' # use this for resolution 100,200
else:
  glacier.config.iceflow_model_lib_path  = 'f14_pism_GJ_21_a'    # use this for resolution 1000,2000

glacier.config.type_mass_balance       = 'signal'
glacier.config.usegpu                  = True

glacier.initialize()

!rm ex.nc ts.nc

glacier.initialize() 
with tf.device(glacier.device_name):
    glacier.load_ncdf_data(glacier.config.geology_file)
    glacier.initialize_fields()               
    while glacier.t < glacier.config.tend:                       
        glacier.update_smb()
        glacier.update_iceflow()
        glacier.update_t_dt() 
        glacier.update_thk()       
        glacier.update_ncdf_ex()
        glacier.update_ncdf_ts()
        glacier.update_plot()
        glacier.print_info()
        
glacier.print_all_comp_info()

Let's vizualize the result as an animation

In [9]:
glacier.animate_result('ex.nc','velsurf_mag',save=True)
#igm.animate_result('ex.nc','velsurf_mag',save=True)

Let's now plot i) the maximum extent with the flowline in plan-view ii) the glacier position along the flowline

In [ ]:
# import flowline and define distance along flowline
fl = np.loadtxt('data-for-paleo-tuto/'+area+'-flowline.dat')
dist = np.cumsum( np.sqrt( np.diff(fl[:,0])**2 + np.diff(fl[:,1])**2 ) )
dist = np.insert(dist,0,0) / 1000

# open IGM's output
nc      = Dataset('ex.nc', 'r')        
time    = nc.variables['time'][:] ;
x       = np.squeeze(nc.variables['x']) ;
y       = np.squeeze(nc.variables['y']) ;
thk     = np.squeeze(nc.variables['thk']) ;
nc.close()

# check what points of the flowiline are covered by ice at time t. 
cov = np.zeros((len(time),len(fl)))
for it in range(len(time)):
    f = RectBivariateSpline(x, y, np.transpose(thk[it]))
    cov[it,:] = f(fl[:,0],fl[:,1],grid=False)>10

# 2D space position along flowline x times
DIST,TIME = np.meshgrid(dist,time)
D = DIST[cov==1]
T = TIME[cov==1]
 
# define the maximum ice thickness
thkmax = np.max(thk,axis=0)
thkmaxnan = np.where( thkmax<1 , np.nan, thkmax)

# plot flowline 
fig, (ax1, ax2) =  plt.subplots(2,1,figsize=(5,5),dpi=200) 
im1 = ax1.imshow(thkmaxnan, origin='lower',  extent=[min(x),max(x),min(y),max(y)])
ax1.plot(fl[:,0],fl[:,1],'-k', linewidth=2)
ax1.set_title('Maximum ice thickness with flowline')
ax1.axis('off') 
ax2.plot(D,T/1000, color='gray', alpha=0.75)
ax2.set(xlabel='Glacier position (km)', ylabel='Timing (ky)')


Now you may try other mass balance parameters (e.g. lowering the ELA to make glacier longer), change the resolution, or try other areas.

In [ ]:
# uncomment this to clean the workspace
!rm -r f12* f14* data-for-paleo-tuto ex-thk.mp4 ex.nc ts.nc igm-run-parameters.txt igm.py computational-statistics.txt